In [ ]:
import pandas as pd
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import re

In [ ]:
symp = '/content/증상정리.csv'      # 자료파일에 있음
symp = pd.read_csv(symp, encoding='euc-kr')

In [ ]:
symp.head()

,Column1,증상
0,45,가래
1,73,가려움
2,51,각기
3,239,감기
4,109,감염


In [ ]:
sym = symp['증상'].tolist()     # 리스트로 만듦

In [ ]:
filename = '/content/key.txt'
with open(filename) as f:
  api_key = f.read()
len(api_key)

88

In [ ]:
med_url = 'http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList'
My_API_Key = api_key

In [ ]:
# sym  = ['두통', '염증', '상처', '과식', '구토', '복통','발열','가려움', '안구건조', '습진', '생리통',
#         '외상','어지러움', '치통', '통증', '진통', '설사', '기침','오한', '근육통', '염좌',
#         '타박상', '신경통', '요통', '어깨결림', '소염', '체함', '화상', '기침',
#         '가래', '속쓰림', '코감기', '변비', '피로', '인후통', '물집', '탈모', '비타민']

In [ ]:
df = pd.DataFrame(columns=['효능','복용법'])         
for s in range(len(sym)):
  queryParams = '?' + urlencode(                                 
      {quote_plus('ServiceKey') : My_API_Key,             
          quote_plus('efcyQesitm') : sym[s] })
  response = requests.get(med_url + queryParams).text.encode('utf-8')
  xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
  name = xmlobj.findAll('itemName')
  eff = xmlobj.findAll('efcyQesitm')
  use = xmlobj.findAll('useMethodQesitm')

  sym1 = []
  for i in range(len(name)):
    med = name[i].get_text().strip()
    sym1.append(med)

  sym1_eff = []
  for i in range(len(eff)):
    med = eff[i].get_text().strip()
    sym1_eff.append(med)

  sym1_eff_list = []
  for i in sym1_eff:
      text = re.sub('(<([^>]+)>)', '' ,i).strip()
      if(text != ''):
        sym1_eff_list.append(text)

  sym1_use = []
  for i in range(len(use)):
    med = use[i].get_text().strip()
    sym1_use.append(med)

  sym1_use_list = []
  for i in sym1_use:
      text = re.sub('(<([^>]+)>)', '' ,i).strip()
      if(text != ''):
        sym1_use_list.append(text)

  df1 = pd.DataFrame(columns=['효능','복용법']) 
  df1['제품명'] = sym1
  df1.효능 = sym1_eff_list
  df1.복용법 = sym1_use_list
  df1['증상'] = sym[s]
  df1 = df1[['증상', '제품명', '효능', '복용법']]

  df = pd.concat([df, df1], axis = 0)
  

In [ ]:
df.head(40)

,효능,복용법,증상,제품명
0,"이 약은 기침, 가래, 천식에 사용합니다.","성인은 1회 10 g, 11~14세는 1회 7 g, 8~10세는 1회 5 g, 5~...",가래,자모
1,"이 약은 기침, 가래에 사용합니다.","성인은 1회 20 mL씩, 11세 이상~15세 미만은 1회 13 mL씩, 8세 이상...",가래,코푸시럽에스
2,"이 약은 감기의 여러 증상(콧물, 코막힘, 재채기, 인후(목구멍)통, 기침, 가래,...","성인 1회 1병(30 mL), 1일 3회 식후 30분에 복용합니다.",가래,판콜에이내복액
3,"이 약은 급·만성기관지염, 기관지천식, 기관지확장증, 인두염의 객담(가래)배출곤란과...","6세 이상~12세 이하는 1회 250 mg을 1일 3회, 2세 이상~5세 이하는 2...",가래,리나치올시럽(카르보시스테인)(수출명:HyundiolSyrup)
4,"이 약은 급·만성기관지염, 기관지천식, 기관지확장증, 인두염의 객담(가래)배출곤란 ...","성인은 초기에 1회 2캡슐(750 mg)씩, 1일 3회 복용하고 증상이 나아지면 1...",가래,리나치올캡슐375밀리그램(카르보시스테인)(수출명:HyundiolCapsule375Mg)
5,"이 약은 급·만성기관지염, 기관지천식, 기관지확장증, 인두염의 객담(가래)배출곤란 ...","성인은 초기에 1회 2캡슐(750 mg)씩, 1일 3회 복용하고 증상이 나아지면 1...",가래,뮤코론캡슐375밀리그램(카르보시스테인)
6,"이 약은 급·만성기관지염, 기관지천식, 기관지확장증, 인두염의 객담(가래)배출곤란과...","성인은 초기에 1회 750 mg씩, 1일 3회 경구투여하고 증상이 나아지면 1회 5...",가래,리나치올시럽5%(카르보시스테인)(수출명:HyundiolSyrup5%)
7,"이 약은 급·만성기관지염, 기관지천식, 기관지확장증, 인두염의 객담(가래)배출곤란 ...",성인은 1회 1캡슐(500 mg)씩 1일 3회 복용합니다.,가래,리나치올캡슐500밀리그람(L-카르보시스테인)(수출명:HyundiolCapsule50...
8,"이 약은 관류, 피부, 창상(상처)면, 점막의 세정, 습포(적셔서 대는 것), 함수...","관류액으로 사용 시에는 의사의 지시에 따라 적량 사용합니다.피부, 창상(상처)면, ...",가래,크린조(염화나트륨)
9,"이 약은 감기의 여러 증상(콧물, 코막힘, 재채기, 기침, 인후(목구멍)통, 가래,...",성인 1회 1병(30 mL) 1일 3회 식후 30분에 복용합니다.,가래,판콜에스내복액


In [ ]:
df.shape

(920, 4)

In [ ]:
df = df[['증상', '제품명', '효능', '복용법']]

In [ ]:
df.to_csv('의약품효능및복용법(update).csv')
df = pd.read_csv('/content/의약품효능및복용법(update).csv')
df.head()

,Unnamed: 0,증상,제품명,효능,복용법
0,0,가래,자모,"이 약은 기침, 가래, 천식에 사용합니다.","성인은 1회 10 g, 11~14세는 1회 7 g, 8~10세는 1회 5 g, 5~..."
1,1,가래,코푸시럽에스,"이 약은 기침, 가래에 사용합니다.","성인은 1회 20 mL씩, 11세 이상~15세 미만은 1회 13 mL씩, 8세 이상..."
2,2,가래,판콜에이내복액,"이 약은 감기의 여러 증상(콧물, 코막힘, 재채기, 인후(목구멍)통, 기침, 가래,...","성인 1회 1병(30 mL), 1일 3회 식후 30분에 복용합니다."
3,3,가래,리나치올시럽(카르보시스테인)(수출명:HyundiolSyrup),"이 약은 급·만성기관지염, 기관지천식, 기관지확장증, 인두염의 객담(가래)배출곤란과...","6세 이상~12세 이하는 1회 250 mg을 1일 3회, 2세 이상~5세 이하는 2..."
4,4,가래,리나치올캡슐375밀리그램(카르보시스테인)(수출명:HyundiolCapsule375Mg),"이 약은 급·만성기관지염, 기관지천식, 기관지확장증, 인두염의 객담(가래)배출곤란 ...","성인은 초기에 1회 2캡슐(750 mg)씩, 1일 3회 복용하고 증상이 나아지면 1..."
